In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import re
from scipy.spatial.distance import pdist, squareform, cdist
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import random
sns.set()
plt.rcParams["figure.figsize"] = [10, 5]

In [ ]:
def tofloat(x):
    try:
        return float(x.replace(',','.').strip()) 
    except:
        return x
    
curves = pd.read_csv(
    'Profiles_BRT_Deriv_Domain1.csv',
    header=None,
    encoding='cp1251',
    index_col=False
)
s = curves.apply(lambda x: x[0].split(';'), axis=1)
f = pd.DataFrame.from_dict(dict(zip(s.index, s.values)), orient='index').fillna('nan')
f = f.applymap(lambda x: x if x != 'nan' and len(x) > 0 else np.nan)
f = f[np.where(f.isna().sum() == 0)[0].tolist()]\
    .applymap(lambda x: tofloat(x))
f['targ'] = f[0].map(lambda x: 1 if '_SCH' in x else 2 if '_Depr' in x else 0)
f = f[f.index % 2 == 0].iloc[:, 1:].reset_index(drop=True)
f = f[(f['targ'] != 2)].reset_index(drop=True)


In [ ]:
distances = cdist(
    f.drop('targ', axis=1),
    f.drop('targ', axis=1),
    metric='minkowski',
    p=1
)
shiz = distances[:50]
healthy = distances[50:]

In [ ]:
df = pd.DataFrame(data=distances, columns=range(69))

In [ ]:
df['target'] = f['targ'].values

In [ ]:
#df.to_csv('matrix.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
df = pd.read_csv('matrix.csv', sep=';', encoding='cp1251')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pwlf
import random
plt.rcParams["figure.figsize"] = [6, 4]

import warnings
warnings.filterwarnings('ignore')

one = []
two = []
pw_rsq = []
lr_rsq = []
p_val = []
p_value = 0.05

def permutation_test_significance(x, y_true, y_pred_s, rdiff, n_permutations=100):
    Nt = 0.0
    resid = y_true - y_pred_s
    for i in range(n_permutations):
        random.shuffle(resid)
        my_pwlf = pwlf.PiecewiseLinFit(x, y_pred_s + resid, lapack_driver='gelsd')
        my_pwlf.fit(2)
        y_pred = my_pwlf.predict(x)
        pwr = r2_score(y_pred_s + resid, y_pred)
        reg = LinearRegression(n_jobs=-1).fit(x.reshape(-1, 1), y_pred_s + resid)
        y_pred = reg.predict(x.reshape(-1, 1))
        lrr = r2_score(y_pred_s + resid, y_pred)
        if abs(pwr - lrr) >= rdiff:
            Nt += 1
    return Nt / n_permutations

for s in range(50):
    for h in range(50, 69):
        one.append(s)
        two.append(h)
        x, y =  shiz[s], healthy[h - 50]

        # initialize piecewise linear fit with your x and y data
        my_pwlf = pwlf.PiecewiseLinFit(x, y, lapack_driver='gelsd')

        # fit the data for four line segments
        res = my_pwlf.fit(2)

        # predict for the determined points
        xHat = np.linspace(min(x), max(x), num=10000)
        yHat = my_pwlf.predict(xHat)

        Rsquared = r2_score(y, my_pwlf.predict(x))
        pw_rsq.append(Rsquared)
  
        reg = LinearRegression(n_jobs=-1).fit(x.reshape(-1, 1), y)
        yHat1 = reg.predict(x.reshape(-1, 1))
        lr_rsq.append(reg.score(x.reshape(-1, 1), y))
        p = permutation_test_significance(x, y, yHat1, abs(pw_rsq[-1] - lr_rsq[-1]))
        p_val.append(p)
        if p < p_value:
            plt.figure()
            plt.scatter(x[:50], y[:50], c='tab:blue', label='schizophrenic')
            plt.scatter(x[50:], y[50:], c='tab:orange', label='healthy')
            plt.legend()
            plt.title('R-squared: {}'.format(round(Rsquared, 2)))
            plt.ylabel('Healthy pacient {}'.format(h))
            plt.xlabel('Schizophrenic pacient {}'.format(s))
            plt.plot(xHat, yHat, '-')
            #plt.savefig('R/{}_{}.png'.format(h, s), dpi=50)
            plt.show()

In [ ]:
from scipy import stats

plt.rcParams["figure.figsize"] = [6, 4]

import warnings
warnings.filterwarnings('ignore')

one = []
two = []
pw_rsq = []
lr_rsq = []
p_val = []
p_value = 0.05
breaks = []
n_sch_in_left = []
n_sch_in_right = []
n_norm_in_left = []
n_norm_in_right = []

for s, d1 in enumerate(distances):
    for h, d2 in enumerate(distances):
        one.append(s)
        two.append(h)
        x, y =  d1, d2
        my_pwlf = pwlf.PiecewiseLinFit(x, y, lapack_driver='gelsd')
        res = my_pwlf.fit(2)
        xHat = np.linspace(min(x), max(x), num=10000)
        yHat = my_pwlf.predict(xHat)
        x_b = my_pwlf.fit_breaks[1]
        y_b = my_pwlf.predict([x_b])[0]
        breaks.append((x_b, y_b))
        Rsquared = r2_score(y, my_pwlf.predict(x))
        pw_rsq.append(Rsquared)
        reg = LinearRegression(n_jobs=-1).fit(x.reshape(-1, 1), y)
        yHat1 = reg.predict(x.reshape(-1, 1))
        lr_rsq.append(reg.score(x.reshape(-1, 1), y))
        matrix = pd.DataFrame(columns=['slope', 'class'])
        matrix['point'] = x
        matrix['slope'] = matrix['point'].map(
            lambda x: 'l' if x < x_b else 'r'
        )
        matrix['class'] = 's'
        matrix['class'].iloc[50:] = 'h'
        confusion_matrix = pd.crosstab(matrix['class'], matrix['slope'])
        p = stats.chi2_contingency(confusion_matrix)[1]
        p_val.append(p)
        '''
        n_sch_in_left.append(len((np.argwhere(x[:50] < x_b)).ravel()))
        n_sch_in_right.append(len((np.argwhere(x[:50] >= x_b)).ravel()))
        n_norm_in_left.append(len((np.argwhere(x[50:] < x_b)).ravel()))
        n_norm_in_right.append(len((np.argwhere(x[50:] >= x_b)).ravel()))
        
        if p < p_value:
            plt.figure()
            plt.scatter(x[:50], y[:50], c='tab:blue', label='schizophrenic')
            plt.scatter(x[50:], y[50:], c='tab:orange', label='healthy')
            plt.scatter([x_b], [y_b], c='black', label='breakpoint ({}, {})'\
                        .format(round(x_b, 2), round(y_b, 2)))
            plt.legend()
            plt.title('R-squared: {}'.format(round(Rsquared, 2)))
            plt.ylabel('Pacient {}'.format(h))
            plt.xlabel('Pacient {}'.format(s))
            plt.plot(xHat, yHat, '-')
            plt.savefig('R1/{}_{}.png'.format(h, s), dpi=50)
            plt.show()
        '''

In [ ]:
from piecewise import piecewise
from piecewise import piecewise_plot

st = pd.read_excel('R1/stats1.xlsx', index_col=0)
ck = st[st['Significance'] == 1].copy()

In [ ]:
t, v =  shiz[6], healthy[62-50]
model = piecewise(t, v, min_stop_frac=1e-7)
piecewise_plot(t, v, model=model)

In [ ]:
import matplotlib.patches as mpatches
plt.rcParams["figure.figsize"] = [6, 4]

for xi, yi in list(zip(ck['One'], ck['Two'])):
    xii, yii = xi, yi
    t, v =  distances[xii], distances[yii]
    model = piecewise(t, v, min_stop_frac=1e-7)
    l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
    x_b = l[0] / (-l[1])
    y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
    try:
        sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
        norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)
    except:
        try:
            sch = random.sample(list(np.argwhere(t[:50] < x_b).ravel()), 10)
            norm = random.sample(list((np.argwhere(t[50:] > x_b) + 50).ravel()), 10) 
        except:
            try:
                t, v = v, t
                xii, yii = yi, xi
                model = piecewise(t, v, min_stop_frac=1e-7)
                l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
                x_b = l[0] / (-l[1])
                y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
                sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
                norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)
            except:
                try:
                    sch = random.sample(list(np.argwhere(t[:50] < x_b).ravel()), 10)
                    norm = random.sample(list((np.argwhere(t[50:] > x_b) + 50).ravel()), 10) 
                except:
                    continue
    for i in sch:
        plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='orange')

    for i, n1 in enumerate(sch):
        for j, n2 in enumerate(sch):
            if i < j and j - i == 1:
                plt.fill_between(
                    range(181),
                    f.drop('targ', axis=1).iloc[n1],
                    f.drop('targ', axis=1).iloc[n2],
                    alpha=0.2,
                    color='orange'
                )

    for i in norm:
        plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='green')

    for i, n1 in enumerate(norm):
        for j, n2 in enumerate(norm):
            if i < j and j - i == 1:
                plt.fill_between(
                    range(181),
                    f.drop('targ', axis=1).iloc[n1],
                    f.drop('targ', axis=1).iloc[n2],
                    alpha=0.2,
                    color='green'
                )

    colors = ["green", "orange"]
    texts = ["Healthy", "Schizophrenic"]
    patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
                label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
    plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
               loc=1, ncol=1, numpoints=1 )
    plt.title('Pair {} - {}'.format(xii, yii))
    #plt.savefig('R1/lines{}_{}.png'.format(xii, yii), dpi=50)
    plt.show()


In [ ]:
import random 
xii, yii = 47, 38
t, v =  distances[xii], distances[yii]#healthy[h-50]
model = piecewise(t, v, min_stop_frac=1e-7)
l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
x_b = l[0] / (-l[1])
y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
try:
    sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
    norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)
except:
    try:
        sch = random.sample(list(np.argwhere(t[:50] < x_b).ravel()), 10)
        norm = random.sample(list((np.argwhere(t[50:] > x_b) + 50).ravel()), 10) 
    except:
        try:
            t, v = v, t
            xii, yii = yii, xii
            model = piecewise(t, v, min_stop_frac=1e-7)
            l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
            x_b = l[0] / (-l[1])
            y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
            sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
            norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)
        except:
            sch = random.sample(list(np.argwhere(t[:50] < x_b).ravel()), 10)
            norm = random.sample(list((np.argwhere(t[50:] > x_b) + 50).ravel()), 10) 
for i in sch:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='gray')

for i, n1 in enumerate(sch):
    for j, n2 in enumerate(sch):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='gray'
            )

for i in norm:
    plt.plot(f.drop('targ', axis=1).iloc[i], '--', color='black')

for i, n1 in enumerate(norm):
    for j, n2 in enumerate(norm):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='black'
            )

colors = ["black", "gray"]
texts = ["Healthy", "Schizophrenic"]
patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]


plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
           loc=1, ncol=1, numpoints=1 )
plt.title('Pair {} - {}'.format(xii, yii))
plt.xlabel('Number of iterations')
plt.savefig('R1/lines{}_{}.png'.format(xii, yii), dpi=50, bbox_inches='tight')
plt.show()


In [ ]:
import matplotlib.patches as mpatches
plt.rcParams["figure.figsize"] = [8, 5]
s, h = 6, 62
t, v =  shiz[0], shiz[37]#healthy[h-50]
model = piecewise(t, v, min_stop_frac=1e-7)
l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
x_b = l[0] / (-l[1])
y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)

for i in sch:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='orange')
    
for i, n1 in enumerate(sch):
    for j, n2 in enumerate(sch):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='orange'
            )
            
for i in norm:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='green')
    
for i, n1 in enumerate(norm):
    for j, n2 in enumerate(norm):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='green'
            )

colors = ["green", "orange"]
texts = ["Healthy", "Schizophrenic"]
patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
           loc=1, ncol=1, numpoints=1 )
plt.title('Pair 0 - 37')
plt.savefig('R/lines{}_{}.png'.format(h, s), dpi=50)
plt.show()

In [ ]:
s, h = 0, 62
t, v =  shiz[s], healthy[h-50]
model = piecewise(t, v, min_stop_frac=1e-7)
l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
x_b = l[0] / (-l[1])
y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)

for i in sch:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='orange')
    
for i, n1 in enumerate(sch):
    for j, n2 in enumerate(sch):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='orange'
            )
            
for i in norm:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='green')
    
for i, n1 in enumerate(norm):
    for j, n2 in enumerate(norm):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='green'
            )

colors = ["green", "orange"]
texts = ["Healthy", "Schizophrenic"]
patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
           loc=1, ncol=1, numpoints=1 )
plt.title('Pair {} - {}'.format(h, s))
plt.savefig('R/lines{}_{}.png'.format(h, s), dpi=50)
plt.show()

In [ ]:
s, h = 8, 64
t, v =  shiz[s], healthy[h-50]
model = piecewise(t, v, min_stop_frac=1e-7)
l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
x_b = l[0] / (-l[1])
y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)

for i in sch:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='orange')
    
for i, n1 in enumerate(sch):
    for j, n2 in enumerate(sch):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='orange'
            )
            
for i in norm:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='green')
    
for i, n1 in enumerate(norm):
    for j, n2 in enumerate(norm):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='green'
            )

colors = ["green", "orange"]
texts = ["Healthy", "Schizophrenic"]
patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
           loc=1, ncol=1, numpoints=1 )
plt.title('Pair {} - {}'.format(h, s))
plt.savefig('R/lines{}_{}.png'.format(h, s), dpi=50)
plt.show()

In [ ]:
s, h = 3, 62
t, v =  shiz[s], healthy[h-50]
model = piecewise(t, v, min_stop_frac=1e-7)
l = np.array(model.segments[0].coeffs) - np.array(model.segments[1].coeffs)
x_b = l[0] / (-l[1])
y_b = model.segments[0].coeffs[0] + model.segments[0].coeffs[1] * x_b
sch = random.sample(list(np.argwhere(t[:50] > x_b).ravel()), 10)
norm = random.sample(list((np.argwhere(t[50:] < x_b) + 50).ravel()), 10)

for i in sch:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='orange')
    
for i, n1 in enumerate(sch):
    for j, n2 in enumerate(sch):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='orange'
            )
            
for i in norm:
    plt.plot(f.drop('targ', axis=1).iloc[i], '-', color='green')
    
for i, n1 in enumerate(norm):
    for j, n2 in enumerate(norm):
        if i < j and j - i == 1:
            plt.fill_between(
                range(181),
                f.drop('targ', axis=1).iloc[n1],
                f.drop('targ', axis=1).iloc[n2],
                alpha=0.2,
                color='green'
            )

colors = ["green", "orange"]
texts = ["Healthy", "Schizophrenic"]
patches = [plt.plot([],[], marker="o", ms=10, ls="", mec=None, color=colors[i], 
            label="{:s}".format(texts[i]) )[0]  for i in range(len(texts)) ]
plt.legend(handles=patches, #bbox_to_anchor=(0.5, 0.5), 
           loc=1, ncol=1, numpoints=1 )
plt.title('Pair {} - {}'.format(h, s))
plt.savefig('R/lines{}_{}.png'.format(h, s), dpi=50)
plt.show()